In [3]:
import psycopg2
import pandas as pd
import os

# === Config ===
DB_NAME = "camping"
DB_USER = "arrowboy"

for file in os.listdir("data"): 
    # === Load CSV ===
    df = pd.read_csv(f'data/{file}')

    # Replace NaNs with None (so psycopg2 sends NULL to SQL)
    df = df.where(pd.notnull(df), None)

    try:
        conn = psycopg2.connect(dbname=DB_NAME, user=DB_USER)
        cur = conn.cursor()

        for _, row in df.iterrows():
            # === Insert into campsites ===
            cur.execute("""
                INSERT INTO campsites (
                    name, latitude, longitude, address, managing_unit,
                    reservation_type, fs_usda_url, reservation_url,
                    contact_name, contact_phone, seasons_of_use,
                    num_sites, fee, overview, site_url
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                RETURNING id;
            """, (
                row['site_name'],
                row['latitude'],
                row['longitude'],
                row['directions'],
                row['info_center'],
                None,                     # reservation_type
                row['park_url'],
                None,                     # reservation_url
                row['info_center'],       # contact_name from info_center
                row['contact_info'],
                row['season_of_use'],
                None,                     # num_sites
                row['fee_info'],
                row['overview'],
                row['site_url']
            ))

            campsite_id = cur.fetchone()[0]  # Get the inserted row's ID

            # === Insert into amenities ===
            cur.execute("""
                INSERT INTO amenities (
                    campsite_id, body_of_water,
                    restrooms, water, amenities_raw
                ) VALUES (%s, %s, %s, %s, %s);
            """, (
                campsite_id,
                row['body_of_water'],  # assumed boolean True/False
                None,
                row['body_of_water'],
                row['amenities']
            ))

        conn.commit()
        print("✅ Data inserted successfully.")

    except Exception as e:
        print("❌ Error inserting data:", e)
        if conn:
            conn.rollback()

    finally:
        if cur:
            cur.close()
        if conn:
            conn.close()

✅ Data inserted successfully.
✅ Data inserted successfully.
✅ Data inserted successfully.
✅ Data inserted successfully.
✅ Data inserted successfully.
✅ Data inserted successfully.
✅ Data inserted successfully.
✅ Data inserted successfully.
✅ Data inserted successfully.
✅ Data inserted successfully.
✅ Data inserted successfully.
✅ Data inserted successfully.
✅ Data inserted successfully.
✅ Data inserted successfully.
✅ Data inserted successfully.
✅ Data inserted successfully.
✅ Data inserted successfully.
✅ Data inserted successfully.
✅ Data inserted successfully.
✅ Data inserted successfully.


In [10]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(dbname="camping", user="arrowboy")
df = pd.read_sql_query("SELECT * FROM campsites", conn)

conn.close()
df.head()
print(df['site_url'][0])

https://www.fs.usda.gov/r05/klamath/recreation/big-flat-trailhead


/tmp/ipykernel_20195/3013866845.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT * FROM campsites", conn)
